In [2]:
import importlib
import ldsc_71 as ld
from helperfuncs import extract_upper_triangle, return_to_symmetric, extract_bounds
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
importlib.reload(ld)

<module 'ldsc_71' from 'C:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\ldsc_71.py'>

In [3]:
N = 100
S = np.array([np.array([[5, 0], [0, 5]]),
    np.array([[2, 0], [0, 2]])] * 50 )# 50 = N/2
V = np.identity(2) * 10.0


model = ld.sibreg_71(S = S)
model.simdata(V, N)

output_matrix, result = model.solve()

Warning there is no value for theta. Maybe consider simulating it
No initial guess provided.
Making a matrix of 0s as the initial estimate
Final Estimate:
 [[ 9.50800236 -0.15483685]
 [-0.15483685 10.01909906]]
Convergence Flag:  b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Number of Iterations:  12
Final Gradient:  [-1.55453091e-06 -2.43050386e-07 -5.36702414e-07]
Max deviation of gradient from 0:  1.5545309138623176e-06


In [32]:
def jackknife(block = 1):
    
    # Simple jackknife estimator for SE
    # Source: https://www.stat.berkeley.edu/~hhuang/STAT152/Jackknife-Bootstrap.pdf
    # Default value of block = 1 is the normal jackknife 
    
    theta = model.theta
    S = model.S
    
    assert theta.shape[0] == S.shape[0]
    
    nobs = theta.shape[0]
    
    estimates_jk = []
    
    start_idx = 0
    while True:
        
        end_idx = start_idx + block
        end_idx_cond = end_idx <= theta.shape[0]
        
        # remove ith observation
        if end_idx_cond:
            theta_jk = np.delete(theta, range(start_idx, end_idx), 
                                 axis = 0)
            S_jk = np.delete(S, range(start_idx, end_idx), 
                             axis = 0)
        else:
            theta_jk = np.delete(theta, range(start_idx, theta.shape[0]), 
                                 axis = 0)
            S_jk = np.delete(S, range(start_idx, S.shape[0]), 
                             axis = 0)
        
        if start_idx < theta.shape[0]:
            # Get our estimate
            output_matrix, _ = model.solve(theta = theta_jk,
                                          S = S_jk,
                                          printout = False)

            estimates_jk.append(output_matrix)

            start_idx +=  block
            if not end_idx_cond:
                break
        
    estimates_jk = np.array(estimates_jk)
    
    # calculate the SE
    estimate_jk_mean = estimates_jk.mean(axis = 0)
    estimate_jk_mean = np.array([estimate_jk_mean] * estimates_jk.shape[0])
    estimates_jk_dev = estimates_jk - estimate_jk_mean
    estimates_jk_devsq = estimates_jk_dev ** 2
    estimates_jk_devsq_sum = estimates_jk_devsq.sum(axis = 0)

    se = ((nobs - 1)/nobs) * estimates_jk_devsq_sum
    se = np.sqrt(se)
    
    return se  

In [34]:
jackknife(1)

array([[1.84732429, 1.19164214],
       [1.19164214, 1.66319927]])